In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from datasets import Dataset
import numpy as np
from bd import nova_conexao
from psycopg2.errors import ProgrammingError
import pandas as pd
import re
import torch

# Obtendo os dados

In [11]:
# Obtendo os dados e colocando-as em um dataframe
sql_select_dados = "select eua.id, eua.curriculo_texto, eva.id, eva.texto, na.nota \
from nota_analise na inner join emprega_usuario_analise eua on na.usuario_id = eua.id \
inner join emprega_vaga_analise eva on na.vaga_id = eva.id \
order by na.vaga_id"
curriculos = []
vagas = []
notas = []
with nova_conexao() as conexao:
    try:
        cursor = conexao.cursor()
        cursor.execute(sql_select_dados)
        resultado = cursor.fetchall()
    except ProgrammingError as e:
        print(f'Erro: {e.msg}')
    else:
        for row in resultado:
            curriculos.append(row[1])
            vagas.append(row[3])
            notas.append(row[4])
        conexao.commit()

df_dados = pd.DataFrame(
    {
        'curriculos' : curriculos,
        'vagas' : vagas,
        'notas' : notas
    })

df_dados

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,1
1,"Rafael Santos Avenida das Estrelas, 789 - Cida...",Desenvolvedor Full Stack Desenvolver soluções ...,2
2,"Amanda Costa Rua das Flores, 123 - Cidade das ...",Desenvolvedor Full Stack Desenvolver soluções ...,5
3,"Leonardo Santos Avenida das Estrelas, 789 - Ci...",Desenvolvedor Full Stack Desenvolver soluções ...,4
4,"Gabriela Lima Rua dos Programadores, 123 - Cid...",Desenvolvedor Full Stack Desenvolver soluções ...,5
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, 1...","Designer Gráfico Criar layouts, peças gráficas...",1
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",1
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",1
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",1


In [12]:
# Salvando para CSV
df_dados.to_csv('dados.csv', index=False, encoding='utf-8')

# Removendo os dígitos

In [13]:
df_dados["curriculos"] = df_dados["curriculos"].apply(lambda x: re.sub('\d+', '', x))
df_dados["vagas"] = df_dados["vagas"].apply(lambda x: re.sub('\d+', '', x))
df_dados

,curriculos,vagas,notas
0,Nome Completo: Ana Paula Santos Formação: B...,Desenvolvedor Full Stack Desenvolver soluções ...,1
1,"Rafael Santos Avenida das Estrelas, - Cidade ...",Desenvolvedor Full Stack Desenvolver soluções ...,2
2,"Amanda Costa Rua das Flores, - Cidade das Art...",Desenvolvedor Full Stack Desenvolver soluções ...,5
3,"Leonardo Santos Avenida das Estrelas, - Cidad...",Desenvolvedor Full Stack Desenvolver soluções ...,4
4,"Gabriela Lima Rua dos Programadores, - Cidade...",Desenvolvedor Full Stack Desenvolver soluções ...,5
...,...,...,...
6195,"Nome: Ana Santos Endereço: Rua das Análises, ...","Designer Gráfico Criar layouts, peças gráficas...",1
6196,Nome: Daniel Oliveira Endereço: Avenida dos Ec...,"Designer Gráfico Criar layouts, peças gráficas...",1
6197,Nome: Pedro Almeida Endereço: Rua das Finanças...,"Designer Gráfico Criar layouts, peças gráficas...",1
6198,Nome: Laura Costa Endereço: Avenida da Economi...,"Designer Gráfico Criar layouts, peças gráficas...",1


# Configurando o Transformer e o Tokenizer

In [14]:
# Transforma em um dataset do HuggingFace para usar a sua função map
dados = Dataset.from_pandas(df_dados)
dados

Dataset({
    features: ['curriculos', 'vagas', 'notas'],
    num_rows: 6200
})

In [15]:
def tokenize_function(inputs):
    return tokenizer(
        inputs['curriculos'], inputs['vagas'], truncation=True, return_tensors='pt', padding='max_length', max_length=128
        )


tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
dados_tokenizados = dados.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=5, hidden_dropout_prob = 0.1, output_hidden_states=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Map:   0%|          | 0/6200 [00:00<?, ? examples/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [16]:
dados_tokenizados = dados_tokenizados.remove_columns(['curriculos', 'vagas'])
dados_tokenizados = dados_tokenizados.rename_column('notas', 'labels')
dados_tokenizados.set_format("torch")

In [17]:
dados_tokenizados

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6200
})

# Preparando o treinamento

In [18]:
train_dataloader = DataLoader(dados_tokenizados, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(dados_tokenizados, batch_size=16)